# softmax 탐험 알고리즘 
#### = Boltzman exploration
#### - 최적의 밴딧을 찾는 또 다른 전략 
- epsilon greedy에서는 최적이 아닌 레버도 공평하게 고려했지만 
- softmax에서는 볼츠만 분포의 확률에 기반하여 레버를 선택 
<img src="./image/BOLTZMAN.PNG">

    - 기호 T(tau)는 온도 계수 (Temperature Factor): 탐색할 수 있는 무작위 레버의 수를 지정 
    - T값이 크면 레버를 똑같이 탐험, 작으면 보상이 높은 레버를 선택

In [1]:
import gym
import gym_bandits
import numpy as np
import math
import random

In [2]:
env = gym.make("BanditTenArmedGaussian-v0")

WARN: Environment '<class 'gym_bandits.bandit.BanditTenArmedGaussian'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.


In [3]:
# 라운드 수 (반복)
num_rounds = 20000

# 레버를 당긴 횟수의 횟수
count = np.zeros(10)

# 각 레버의 보상의 합계
sum_rewards = np.zeros(10)

# 평균 보상인 Q값
Q = np.zeros(10)

env.reset()

0

- softmax함수 정의

In [9]:
def softmax(tau):
    
    
    total = sum([math.exp(val/tau) for val in Q])    
    probs = [math.exp(val/tau)/total for val in Q]
    
    threshold = random.random()
    cumulative_prob = 0.0
    
    for i in range(len(probs)):
        cumulative_prob += probs[i]
        
        if (cumulative_prob > threshold):
            return i 
        
    return np.argmax(probs) 

In [14]:
tau = 0.1
[math.exp(val/tau) for val in Q]

[105.57836060407467,
 6.6847805368955076,
 3111.6221381851406,
 60932866.378458716,
 1.2617533946910413e-06,
 0.004859612850571469,
 14.897169033681932,
 0.003909713082697649,
 60454.8406653637,
 35680.61005109552]

In [13]:
tau = 100
[math.exp(val/tau) for val in Q]

[1.0046703255643994,
 1.001901639207676,
 1.0080753304610008,
 1.0180869054183348,
 0.9865088245761433,
 0.9946873657182967,
 1.0027048226470685,
 0.9944710499297273,
 1.011070481179148,
 1.0105374951191142]

- 레버를 당겨보자 

In [16]:
for i in range(num_rounds):
    
    # 소프트맥스를 사용하여 레버 선택
    arm = softmax(0.5)

    observation, reward, done, info = env.step(arm) 

    count[arm] += 1
    
    sum_rewards[arm]+=reward
    
    # 레버의 평균보상인 Q값을 계산
    Q[arm] = sum_rewards[arm]/count[arm]
    
print( 'The optimal arm is {}'.format(np.argmax(Q)))

#탐험시에 Q값이 큰 소프트맥스 적용

The optimal arm is 3


In [17]:
tau = 0.1
[math.exp(val/tau) for val in Q]

[132.7045133617458,
 5.745320420132888,
 3253.7849968861515,
 60743686.592971675,
 2.137402484821704e-07,
 0.005674575220385765,
 9.647413246184161,
 0.018306458898477113,
 52320.53465982411,
 30143.34151605115]

In [18]:
tau = 100
[math.exp(val/tau) for val in Q]

[1.0049000913250679,
 1.0017499150008828,
 1.0081203669854801,
 1.0180837396265137,
 0.9847588359845649,
 0.9948415909539245,
 1.0022692607054997,
 0.9960074900093225,
 1.0109243842405196,
 1.0103670890580798]